# Resumão Limpeza e Tratamento de Dados


## Importando bibliotecas e lendo o arquivo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import statistics  as sts

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/Coisas em python/Udemy/Analise de Dados/dados/Churn.csv", sep=";")

Arrumando o nome das colunas

In [ ]:
dataset.columns = ["Id","Score","Estado","Genero","Idade","Patrimonio","Saldo","Produtos","TemCartCredito",
                    "Ativo","Salario","Saiu"]

## Vendo por cima o arquivo

Para visualizar as dimensões do arquivo, basta fazer o seguinte comando:

In [ ]:
dataset.shape

(999, 12)

Para ver o arquivo propriamente dito, apenas colocamos o nome do arquivo na linha de comando, ou digitamos ```DataFrame.head()```.

In [ ]:
dataset

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
0,1,619,RS,Feminino,42,2,0,1,1,1,10134888.0,1
1,2,608,SC,Feminino,41,1,8380786,1,0,1,11254258.0,0
2,3,502,RS,Feminino,42,8,1596608,3,1,0,11393157.0,1
3,4,699,RS,Feminino,39,1,0,2,0,0,9382663.0,0
4,5,850,SC,Feminino,43,2,12551082,1,1,1,790841.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
994,996,838,SC,Masculino,43,9,12310588,2,1,0,14576583.0,0
995,997,610,SC,Masculino,29,9,0,3,0,1,8391224.0,0
996,998,811,SC,Masculino,44,3,0,2,0,1,7843973.0,0
997,999,587,SC,Masculino,62,7,12128627,1,0,1,677692.0,0


In [ ]:
dataset.head()

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
0,1,619,RS,Feminino,42,2,0,1,1,1,10134888.0,1
1,2,608,SC,Feminino,41,1,8380786,1,0,1,11254258.0,0
2,3,502,RS,Feminino,42,8,1596608,3,1,0,11393157.0,1
3,4,699,RS,Feminino,39,1,0,2,0,0,9382663.0,0
4,5,850,SC,Feminino,43,2,12551082,1,1,1,790841.0,0


## Checando as colunas

### Colunas de string

Como temos apenas caracteres alfabéticos, o que podemos checar é a frequência com que um elemento aparece. 

Para isso, usamos ```.groupby([Nome da coluna]).size()```.

Exemplo:

In [ ]:
dataset.groupby(['Estado']).size()

Estado
PR    257
RP      1
RS    478
SC    258
SP      4
TD      1
dtype: int64

### Colunas numéricas

Já para colunas numéricas, podemos extrair valores como média, valor máximo e mínimo de uma determinada coluna. 

Para visualizar isso, usamos ```DataFrame[Nome da coluna].describe()```.

Exemplo:

In [ ]:
dataset['Idade'].describe()

count    999.000000
mean      38.902903
std       11.401912
min      -20.000000
25%       32.000000
50%       37.000000
75%       44.000000
max      140.000000
Name: Idade, dtype: float64

## Falta de Padrão

Para arrumar a falta de padrão, vamos primeiro dar uma olhada no que está acontecendo na coluna 'Genero'

In [ ]:
dataset.groupby(['Genero']).size()

Genero
F              2
Fem            1
Feminino     461
M              6
Masculino    521
dtype: int64

Para arrumar todos os M's para Masculino, basta fazer

In [ ]:
dataset.loc[dataset['Genero'] ==  'M', 'Genero'] = 'Masculino'

Já para arrumar o caso feminino, podemos fazer isso em uma linha com

In [ ]:
dataset.loc[ dataset['Genero'].isin( ['Fem','F'] ), 'Genero' ] = 'Feminino'

##**Células nulas (NaN)**

No caso de colunas de string, o mais normal é substituir as células não nulas pelo elemento que aparece mais vezes (moda). No caso da coluna 'Genero', a moda é 'Masculino'

In [ ]:
dataset.groupby(['Genero']).size()

Genero
Feminino     464
Masculino    527
dtype: int64

Assim, substituímos os NaN's por Masculino, mas antes disso, vejamos quantas céulas estão vazias com o seguinte comando:

In [ ]:
dataset['Genero'].isnull().sum()

8

Para substituir essas 8 lacunas, usamos apenas

In [ ]:
dataset['Genero'].fillna('Masculino', inplace=True)

Já no caso de colunas numéricas, fazemos os mesmos comandos só que substituímos as lacunas pela média ou pela mediana. 

Por exemplo,

In [ ]:
mediana = sts.median(dataset['Salario'])

para calcular a mediana dos valores contidos numa coluna, e finalmente

In [ ]:
dataset['Salario'].fillna(mediana, inplace=True)

## Linhas duplicadas

### Introdução

Vamos nesta seção resolver o problema dos dados duplicados. Para encontrá-los, introduzimos a função ```.duplicated()```, que nos retorna uma série onde os elementos repetidos são True e os demais são False.

In [ ]:
dataset.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
994    False
995    False
996    False
997    False
998    False
Length: 999, dtype: bool

Assim, para ter a linha completa do dado repetido, inserimos esse comando dentro dos colchetes em ```dataset[]```

In [ ]:
dataset[dataset.duplicated()]

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
81,81,665,RS,Feminino,34,1,9664554,2,0,0,17141366.0,0


Mas onde estão as demais linhas duplicadas? Ocorre que essa função possui um parâmetro chamado 'keep', que por padrão está deixado em 'first', o que significa que ele "vai passar reto" na primeira vez que o elemento repetido aparecer. 

Para ver todos as linhas repetidas, configuramos esse padrão como 'False'.

In [ ]:
dataset[dataset.duplicated(keep=False)]

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
80,81,665,RS,Feminino,34,1,9664554,2,0,0,17141366.0,0
81,81,665,RS,Feminino,34,1,9664554,2,0,0,17141366.0,0


### Excluindo dados duplicados

Para excluir os dados duplicados, usamos a função ```.drop_duplicates()```, e colocamos os parâmetros ```keep = 'first'```, para manter o primeiro elemento repetido e excluir as demais, e ```inplace = True``` por padrão.

In [ ]:
dataset.drop_duplicates(keep='first',inplace=True)

Checando se de fato as linhas repetidas foram excluídas.

In [ ]:
dataset[dataset.duplicated(keep=False)]

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
